In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import lightgbm as lgb
import os
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
#import pandas_profiling

In [53]:
# ## plotly 
# import plotly.plotly as py
# import plotly.graph_objs as go
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# init_notebook_mode(connected=True)

In [78]:
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:50].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    #plt.savefig('lgbm_importances01.png')

In [102]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

## FE

In [106]:
train_df.tail(10)

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190_exp,var_191_exp,var_192_exp,var_193_exp,var_194_exp,var_195_exp,var_196_exp,var_197_exp,var_198_exp,var_199_exp
199990,train_199990,1,14.1475,1.8568,11.0066,3.6779,12.1944,-16.5936,5.3217,14.8508,...,0.001548,113.148374,6.708665,1383.539016,8.881933e+08,5.567800,0.018257,9534.766223,9.922199e+05,1.402120e-01
199991,train_199991,0,9.9909,2.5523,11.9653,6.3958,13.5497,-9.5293,6.0864,14.1789,...,175202.898273,183.809519,2.246560,0.517627,2.551559e+07,1.703696,0.004775,5122.510859,5.909579e+05,1.125419e-03
199992,train_199992,0,12.2825,2.6918,15.4684,6.4262,10.9863,9.9659,4.5030,9.9232,...,52.008127,54393.503241,3.846640,275.448313,8.407459e+08,0.751714,2049.190262,26259.943597,3.627357e+07,9.885910e-06
199993,train_199993,0,13.2152,-5.8006,9.7260,6.5910,12.4612,-7.1652,6.0666,12.9887,...,51.197976,6934.054597,6.108004,0.007975,3.602882e+09,1.479346,837.314712,7870.682033,2.064709e+06,4.525508e+02
199994,train_199994,0,12.3925,-5.8821,11.2323,3.9237,10.4509,10.7262,7.0503,18.6968,...,4.019269,961.697546,43.763494,565.550730,1.915712e+06,0.226661,0.019646,9103.370422,1.220065e+07,5.504359e-04
199995,train_199995,0,11.4880,-0.4956,8.2622,3.5142,10.3404,11.6081,5.6709,15.1516,...,464.750174,557099.979939,54.060295,2.556911,6.731543e+07,0.166310,8.724193,5077.630527,1.729625e+07,1.741211e-08
199996,train_199996,0,4.9149,-2.4484,16.7052,6.6345,8.3096,-10.5628,5.8802,21.5940,...,142.750735,105.098710,2.013350,6.259498,4.704099e+09,5.661563,0.114738,847.168828,8.084021e+06,1.403263e+00
199997,train_199997,0,11.2232,-5.0518,10.5127,5.6456,9.3410,-5.4086,4.5555,21.5571,...,58.270735,230.765028,22.269099,131.538555,1.657398e+10,0.207298,3.608167,6096.682197,1.017542e+06,6.665300e+01
199998,train_199998,0,9.7148,-8.6098,13.6104,5.7930,12.5173,0.5339,6.0479,17.0152,...,14.643551,5760.041739,15.389724,67359.552130,7.353080e+08,0.924410,896.053392,22792.800556,5.547733e+06,9.188895e-07
199999,train_199999,0,10.8762,-5.7105,12.1183,8.0328,11.5577,0.3488,5.2839,15.2058,...,7976.061323,5.415688,1.136098,1.457321,4.033318e+06,0.082852,0.107078,3589.255219,1.850535e+05,1.148550e+00


In [104]:
var_list = [f for f in train_df.columns if 'var' in f]
extra_var_list = [f+'_exp' for f in var_list]

In [107]:

train_df[extra_var_list] = np.exp(train_df[var_list])
test_df[extra_var_list] = np.exp(test_df[var_list])


In [108]:
test_df.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190_exp,var_191_exp,var_192_exp,var_193_exp,var_194_exp,var_195_exp,var_196_exp,var_197_exp,var_198_exp,var_199_exp
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,...,0.115834,140014.322510,0.239309,11.597621,9.009455e+05,11.785854,78.680866,45251.902842,5.241928e+06,1.633362e-04
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,...,40802.555776,6869.866825,2.560750,25039.253188,5.818191e+06,1.611717,0.226457,19368.435812,2.031185e+08,7.766743e-10
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,...,0.473123,59486.221899,7.244916,8.846306,4.342171e+05,8.398894,0.000818,1166.543055,4.370683e+08,8.576587e-11
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,...,14331.282294,8748.171622,5.248803,35.920207,3.942793e+06,23.702962,52.284502,10193.443516,4.499087e+05,1.483450e-02
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,...,68.436068,9626.740751,3.609250,29.306226,3.106570e+08,0.751263,0.005735,1462.935043,1.116825e+06,1.026075e-04


## Model

In [109]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

n_folds = 5
random_seed = 26
model = 'exp'


model_name = "{0}_{1}_folds".format(model, n_folds)
print("Model: {}".format(model_name))

Model: exp_5_folds


In [110]:
#feats = train_df.iloc[:,2:].columns.tolist();feats

#exclusion = ['ID_code', 'target']+ rank_var_list
exclusion = ['ID_code', 'target'] + var_list

feats = [c for c in train_df.columns if c not in exclusion]

#cat_feats = [c for c in feats if c not in var_list]

In [111]:
feats

['var_0_exp',
 'var_1_exp',
 'var_2_exp',
 'var_3_exp',
 'var_4_exp',
 'var_5_exp',
 'var_6_exp',
 'var_7_exp',
 'var_8_exp',
 'var_9_exp',
 'var_10_exp',
 'var_11_exp',
 'var_12_exp',
 'var_13_exp',
 'var_14_exp',
 'var_15_exp',
 'var_16_exp',
 'var_17_exp',
 'var_18_exp',
 'var_19_exp',
 'var_20_exp',
 'var_21_exp',
 'var_22_exp',
 'var_23_exp',
 'var_24_exp',
 'var_25_exp',
 'var_26_exp',
 'var_27_exp',
 'var_28_exp',
 'var_29_exp',
 'var_30_exp',
 'var_31_exp',
 'var_32_exp',
 'var_33_exp',
 'var_34_exp',
 'var_35_exp',
 'var_36_exp',
 'var_37_exp',
 'var_38_exp',
 'var_39_exp',
 'var_40_exp',
 'var_41_exp',
 'var_42_exp',
 'var_43_exp',
 'var_44_exp',
 'var_45_exp',
 'var_46_exp',
 'var_47_exp',
 'var_48_exp',
 'var_49_exp',
 'var_50_exp',
 'var_51_exp',
 'var_52_exp',
 'var_53_exp',
 'var_54_exp',
 'var_55_exp',
 'var_56_exp',
 'var_57_exp',
 'var_58_exp',
 'var_59_exp',
 'var_60_exp',
 'var_61_exp',
 'var_62_exp',
 'var_63_exp',
 'var_64_exp',
 'var_65_exp',
 'var_66_exp',
 'var

In [ ]:
clfs = []
folds = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_seed)
oof_preds = np.zeros((len(train_df), 1))
test_preds = np.zeros((len(test_df), 1))


X = train_df[feats]
y = train_df['target']
X_test = test_df[feats]
test_ids = test_df.ID_code.values


parameters = {
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    #'scale_pos_weight': 400,
    #'device' : 'gpu' ,
    'boosting': 'gbdt',
    'num_leaves': 31, #31
    'feature_fraction': 0.5,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'learning_rate': 0.05, #0.05
    'verbose': 30
    #'min_data_in_leaf': 200
}

feature_importance_df = pd.DataFrame()
for fold_, (trn_, val_) in enumerate(folds.split(X, y)):
    print("Current Fold: {}".format(fold_+1))
    trn_x, trn_y = X.iloc[trn_, :], y[trn_]
    val_x, val_y = X.iloc[val_, :], y[val_]


    trn_lgb = lgb.Dataset(trn_x, trn_y)
    val_lgb = lgb.Dataset(val_x, val_y)
    clf = lgb.train(parameters,
                     train_set=trn_lgb,
                     #valid_sets=[valid_data_lgb,holdout_data_lgb],
                     valid_sets=[trn_lgb, val_lgb],
                     num_boost_round=3000,
                     early_stopping_rounds=50,
                     verbose_eval=False)
    


    val_pred = clf.predict(val_x)
    test_fold_pred = clf.predict(X_test)

    print("AUC = {}".format(roc_auc_score(val_y, val_pred)))
    oof_preds[val_, :] = val_pred.reshape((-1, 1))
    test_preds += test_fold_pred.reshape((-1, 1))
    
    print('getting feature importance')
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = feats
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    
test_preds /= n_folds
roc_score = roc_auc_score(y, oof_preds.ravel())
print("Overall AUC = {}".format(roc_score))


print("Saving submission file")
sample = pd.read_csv('../data/sample_submission.csv')
sample.target = test_preds.astype(float)
sample.ID_code = test_ids
sample.to_csv('../submissions/{}_{}.csv'.format(model_name,str(roc_score)), index=False)

display_importances(feature_importance_df)


Current Fold: 1
AUC = 0.8916842443517385
getting feature importance
Current Fold: 2
AUC = 0.8920239620662148
getting feature importance
Current Fold: 3
AUC = 0.8948401613389418
getting feature importance
Current Fold: 4
AUC = 0.8949511706553405
getting feature importance
Current Fold: 5


In [95]:
%%javascript
var nb = IPython.notebook;
var kernel = IPython.notebook.kernel;
var command = "NOTEBOOK_FULL_PATH = '" + nb.base_url + nb.notebook_path + "'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [96]:


shutil.copyfile(os.path.basename(NOTEBOOK_FULL_PATH), 
                             '../models/{}_{}.ipynb'.format(model_name, str(roc_score)))


'../models/pt_and_minmax_scale_5_folds_0.8929245958292267.ipynb'